In [1]:
from utils import load_dataset, fill_mask, split_dataset, bool_ext

/home/ksuga/.pyenv/versions/3.7.0/envs/attention/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
datset,ptw_ids = load_dataset(input_dir = "data/input", repository = "gdsc", drug_id = -1)

In [7]:
train_set,test_set = split_dataset(datset,ratio=0.8)

In [10]:
train_set["tgt"],train_set['msk'] = fill_mask(train_set['tgt'],train_set['msk'])

In [11]:
train_set['tgt'],train_set['msk']

(array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]),
 array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]))

In [13]:
len(train_set['msk'])

676